In [20]:
from src.codes import ENCODERS, NonsystematicEncoders, SystematicEncoders, SystematicDecoders, NonsystematicDecoders, DECODERS
from src.dataclasses import NonsystematicTurboEncoderSpec, SystematicTurboEncoderSpec
from src.channels import CHANNELS, NoisyChannels
from src.channelcoding.interleavers import FixedPermuteInterleaver
from src.channelcoding.codes import ProjectionCode
from src.channelcoding.bcjr import (BCJRDecoder, SystematicTurboRepeater, TurboDecoder)
from src.channelcoding.metrics import cross_entropy_with_logits
from src.utils import sigma2snr, snr2sigma
from src.channelcoding.utils import enumerate_binary_inputs
from dataclasses import asdict
from pprint import pprint
from typing import cast
import math

import numpy as np
import tensorflow as tf

In [3]:
def entropy_estimate(log_conditional_density_matrix, indices):
    # log_conditional_density_matrix is Received=m x Sent=m
    # indices is 1D array of subset of [m]
    """
    Suppose we have random variable Y, X, U w/ joint distribution. Suppose we sample {(y_i, x_i, u_i)| i =1 to n}. 
    Then the log conditional density matrix entry (i,j) is ln f_(Y|X = x_j)(y_i). That is, the conditional density 
    of Y given that X = x_j evaluated at y_i. If were to take exp and take expectation over all choices for X, then 
    I would get f_Y(y_i), the density of Y evaluated at y_i. Specifying the indices, allows me to get the density 
    conditioned on X being in some subset of its values specified by the choice of indicies. Usually we want to
    restrict indices based on some relationship between X and U we want satisfied (e.g U=1)
    """
    filtered_log_conditional_density_matrix = tf.gather(tf.gather(log_conditional_density_matrix, indices, axis=1), indices, axis=0)
    num_elems = tf.cast(tf.size(indices), tf.float32)
    log_density_estimate = tf.math.reduce_logsumexp(filtered_log_conditional_density_matrix, axis=1)
    return tf.math.log(num_elems) - 1 / num_elems * tf.math.reduce_sum(log_density_estimate, axis=0)
    
def mean_conditional_entropy_estimate(log_conditional_density_matrix, inputs):
    # inputs is 2D array of m x n inputs. m is # of samples, n is # of bits
    # Requires that for each bit there be an input with a 1 and one with a 0.
    m, n = inputs.shape
    entropys = tf.TensorArray(dtype=tf.float32, size=n, clear_after_read=True)
    for i in tf.range(n):
        bool_1 = inputs[:, i] == 1.
        bool_0 = ~bool_1
        entropy_0 = entropy_estimate(log_conditional_density_matrix, tf.reshape(tf.where(bool_0), (-1)))
        entropy_1 = entropy_estimate(log_conditional_density_matrix, tf.reshape(tf.where(bool_1), (-1)))
        entropys = entropys.write(i, 0.5 * entropy_0 + 0.5 * entropy_1)  # Hardcoded to think inputs are uniform dist
    return tf.reduce_mean(entropys.stack())

def get_squared_distance_matrix(Y, X):
    """
    both Y and X are m x n matrices where m is number of samples and n is number of features.
    This computes the distance between the Y_i and X_j
    """
    # Inner products
    i_prods = Y @ tf.transpose(X, perm=(1, 0))  # Matrix Multiplication
    y_norms = tf.square(tf.norm(Y, axis=1, keepdims=True))
    x_norms_T = tf.square(tf.transpose(tf.norm(X, axis=1, keepdims=True), perm=(1, 0)))
    return y_norms - 2 * i_prods + x_norms_T
    

def get_gaussian_log_cond_matrix(Y, X, variance):
    squared_distance_matrix = get_squared_distance_matrix(Y, X)
    n = tf.cast(Y.shape[1], dtype=tf.float32)
    return (-n / 2) * tf.math.log(2 * math.pi * variance) - (1 / variance) * squared_distance_matrix

In [18]:

# encoder_spec

message_length = 10
interleaver = FixedPermuteInterleaver(message_length)

channel_factory = CHANNELS[NoisyChannels.AWGN]
channel_factory

<function src.channels.awgn(sigma, **kwargs) -> src.channelcoding.channels.AWGN>

In [24]:
systematic = False

repeats = 10
all_possible_inputs = tf.cast(enumerate_binary_inputs(message_length), dtype=tf.float32)
print(all_possible_inputs.shape)
sample_size = repeats * all_possible_inputs.shape[0]
# sample_size = 10000
# messages = tf.cast(tf.random.uniform((sample_size, message_length), minval=0, maxval=2, dtype=tf.int32), tf.float32)
messages = tf.repeat(all_possible_inputs, repeats=repeats, axis=0)
assert messages.shape[0] == sample_size
print(f"Sample Size {sample_size}")

snr = -1.5
sigma = snr2sigma(snr)
print(f"Using snr: {sigma2snr(sigma)}")
print(f"Using sigma: {sigma}")
variance = sigma ** 2
channel = channel_factory(sigma=sigma, block_len=message_length)

if not systematic:
    print("using nonsystematic codes")
    ##### Nonsystematic Codes - Start
    # encoder_name = NonsystematicEncoders.TURBOAE_APPROXIMATED_NONSYS
    encoder_name = NonsystematicEncoders.TURBOAE_BINARY_EXACT
    encoder_spec = cast(NonsystematicTurboEncoderSpec, ENCODERS[encoder_name]())

    print(f"Interleaver: {interleaver.permutation}")
    print(f"Encoder {encoder_name}")
    print(encoder_spec)

    ### This is whole code with interleaver
    # encoder = encoder_spec.noninterleaved_code \
    #     .concat(
    #         interleaver.and_then(encoder_spec.interleaved_code)
    #     )
    
    # non_interleaved_bcjr = BCJRDecoder(
    #     encoder_spec.noninterleaved_code,
    #     channel, use_max=False
    # )
    # interleaved_bcjr = BCJRDecoder(
    #     encoder_spec.interleaved_code,
    #     channel, use_max=False
    # )

    # decoder = DECODERS[NonsystematicDecoders.BASIC](
    #     decoder1=non_interleaved_bcjr,
    #     decoder2=interleaved_bcjr,
    #     interleaver=interleaver,
    #     num_iter=6
    # )
    
    ### This one is just noninterleaved code
    print("Using just noninterleaved code")
    encoder = encoder_spec.noninterleaved_code
    
    decoder = BCJRDecoder(
        encoder_spec.noninterleaved_code,
        channel, use_max=False
    )
    ##### Nonsystematic Codes - End

if systematic:
    print("Using systematic codes")
    ###### Systematic codes
    encoder_name = SystematicEncoders.TURBOAE_APPROXIMATED_RSC2
    # encoder_name = SystematicEncoders.TURBOAE_BINARY_EXACT_RSC
    encoder_spec = cast(SystematicTurboEncoderSpec, ENCODERS[encoder_name]())

    print(f"Interleaver: {interleaver.permutation}")
    print(f"Encoder {encoder_name}")
    print(encoder_spec)
    interleaved_code_with_systematic = encoder_spec.interleaved_code.with_systematic()

    ### This is whole code with interleaver
    encoder = encoder_spec.systematic_code \
                .concat(
                    interleaver.and_then(interleaved_code_with_systematic).and_then(ProjectionCode((1,)))
                )

    ### This is just systematic stream
    # encoder = encoder_spec.systematic_code

    non_interleaved_bcjr = BCJRDecoder(
                # systematic_code.trellis,
                encoder_spec.systematic_code,
                channel, use_max=False
            )
    interleaved_bcjr = BCJRDecoder(
            # interleaved_code.trellis.with_systematic(),
            interleaved_code_with_systematic,
            channel, use_max=False
        )
    repeater = SystematicTurboRepeater(
        num_noninterleaved_streams=non_interleaved_bcjr.num_input_channels, 
        interleaver=interleaver
    )
    decoder = repeater.and_then(
        DECODERS[SystematicDecoders.HAZZYS](
            decoder1=non_interleaved_bcjr,
            decoder2=interleaved_bcjr,
            interleaver=interleaver,
            num_iter=6
        )
    )

###### Systematic Codes - End

encoded_messages = encoder(messages[..., None])  # Add a channel axis
received_messages = channel(encoded_messages)
decoded_confidence = decoder(received_messages)
cross_entropy = tf.reduce_mean(cross_entropy_with_logits(messages[..., None], decoded_confidence)["cross_entropy"])
print(f"Cross Entropy: {cross_entropy}")

flat_encoded_messages = tf.reshape(encoded_messages, (sample_size, -1))
flat_received_messages = tf.reshape(received_messages, (sample_size, -1))


gaussian_log_cond_matrix = get_gaussian_log_cond_matrix(flat_received_messages, flat_encoded_messages, variance=variance)
constellation_entropy = entropy_estimate(gaussian_log_cond_matrix, indices=tf.range(sample_size))
conditional_entropy = mean_conditional_entropy_estimate(gaussian_log_cond_matrix, messages)
input_entropy = tf.math.log(2.)
# input_entropy_estimate = 

print(f"Constellation Entropy: {constellation_entropy}")
print(f"Mean Conditional Entropy: {conditional_entropy}")
print(f"Input entropy: {input_entropy}")

entropy_score_estimate = conditional_entropy + input_entropy - constellation_entropy
print(f"Entropy score estimate: {entropy_score_estimate}")

kl_divergence_estimate = cross_entropy - entropy_score_estimate
print(f"Decoder KL Divergence estimate: {kl_divergence_estimate}")

(1024, 10)
Sample Size 10240
Using snr: -1.5000000000000002
Using sigma: 1.1885022274370185
using nonsystematic codes
Interleaver: [3 9 0 6 4 2 5 8 1 7]
Encoder turboae-binary-exact
NonsystematicTurboEncoderSpec(noninterleaved_code=<src.channelcoding.encoders.GeneralizedConvolutionalCode object at 0x7f03002474f0>, interleaved_code=<src.channelcoding.encoders.GeneralizedConvolutionalCode object at 0x7f033ec9b670>)
Using just noninterleaved code
Cross Entropy: 1.2406785488128662
Constellation Entropy: 53.258453369140625
Mean Conditional Entropy: 53.0228385925293
Input entropy: 0.6931471824645996
Entropy score estimate: 0.4575309753417969
Decoder KL Divergence estimate: 0.7831475734710693


In [17]:
systematic = True

sample_size = 10000
messages = tf.cast(tf.random.uniform((sample_size, message_length), minval=0, maxval=2, dtype=tf.int32), tf.float32)

snr = -1.5
sigma = snr2sigma(snr)
print(f"Using snr: {sigma2snr(sigma)}")
print(f"Using sigma: {sigma}")
variance = sigma ** 2
channel = channel_factory(sigma=sigma, block_len=message_length)

if not systematic:
    print("using nonsystematic codes")
    ##### Nonsystematic Codes - Start
    encoder_name = NonsystematicEncoders.TURBOAE_APPROXIMATED_NONSYS
    # encoder_name = NonsystematicEncoders.TURBOAE_BINARY_EXACT
    encoder_spec = cast(NonsystematicTurboEncoderSpec, ENCODERS[encoder_name]())

    print(f"Interleaver: {interleaver.permutation}")
    print(f"Encoder {encoder_name}")
    print(encoder_spec)

    ### This is whole code with interleaver
    encoder = encoder_spec.noninterleaved_code \
        .concat(
            interleaver.and_then(encoder_spec.interleaved_code)
        )

    ### This one is just noninterleaved code
    # encoder = encoder_spec.noninterleaved_code
    
    non_interleaved_bcjr = BCJRDecoder(
        encoder_spec.noninterleaved_code,
        channel, use_max=False
    )
    interleaved_bcjr = BCJRDecoder(
        encoder_spec.interleaved_code,
        channel, use_max=False
    )

    decoder = DECODERS[NonsystematicDecoders.BASIC](
        decoder1=non_interleaved_bcjr,
        decoder2=interleaved_bcjr,
        interleaver=interleaver,
        num_iter=6
    )
    ##### Nonsystematic Codes - End

if systematic:
    print("Using systematic codes")
    ###### Systematic codes
    # encoder_name = SystematicEncoders.TURBOAE_APPROXIMATED_RSC2
    # encoder_name = SystematicEncoders.TURBOAE_BINARY_EXACT_RSC
    encoder_name = SystematicEncoders.TURBO_155_7
    encoder_spec = cast(SystematicTurboEncoderSpec, ENCODERS[encoder_name]())

    print(f"Interleaver: {interleaver.permutation}")
    print(f"Encoder {encoder_name}")
    print(encoder_spec)
    interleaved_code_with_systematic = encoder_spec.interleaved_code.with_systematic()

    ### This is whole code with interleaver
    encoder = encoder_spec.systematic_code \
                .concat(
                    interleaver.and_then(interleaved_code_with_systematic).and_then(ProjectionCode((1,)))
                )

    ### This is just systematic stream
    # encoder = encoder_spec.systematic_code

    non_interleaved_bcjr = BCJRDecoder(
                # systematic_code.trellis,
                encoder_spec.systematic_code,
                channel, use_max=False
            )
    interleaved_bcjr = BCJRDecoder(
            # interleaved_code.trellis.with_systematic(),
            interleaved_code_with_systematic,
            channel, use_max=False
        )
    repeater = SystematicTurboRepeater(
        num_noninterleaved_streams=non_interleaved_bcjr.num_input_channels, 
        interleaver=interleaver
    )
    decoder = repeater.and_then(
        DECODERS[SystematicDecoders.HAZZYS](
            decoder1=non_interleaved_bcjr,
            decoder2=interleaved_bcjr,
            interleaver=interleaver,
            num_iter=6
        )
    )

###### Systematic Codes - End

encoded_messages = encoder(messages[..., None])  # Add a channel axis
received_messages = channel(encoded_messages)
decoded_confidence = decoder(received_messages)
cross_entropy = tf.reduce_mean(cross_entropy_with_logits(messages[..., None], decoded_confidence)["cross_entropy"])
print(f"Cross Entropy: {cross_entropy}")

flat_encoded_messages = tf.reshape(encoded_messages, (sample_size, -1))
flat_received_messages = tf.reshape(received_messages, (sample_size, -1))


gaussian_log_cond_matrix = get_gaussian_log_cond_matrix(flat_received_messages, flat_encoded_messages, variance=variance)
constellation_entropy = entropy_estimate(gaussian_log_cond_matrix, indices=tf.range(sample_size))
conditional_entropy = mean_conditional_entropy_estimate(gaussian_log_cond_matrix, messages)
input_entropy = tf.math.log(2.)
# input_entropy_estimate = 

print(f"Constellation Entropy: {constellation_entropy}")
print(f"Mean Conditional Entropy: {conditional_entropy}")
print(f"Input entropy: {input_entropy}")

entropy_score_estimate = conditional_entropy + input_entropy - constellation_entropy
print(f"Entropy score estimate: {entropy_score_estimate}")

kl_divergence_estimate = cross_entropy - entropy_score_estimate
print(f"Decoder KL Divergence estimate: {kl_divergence_estimate}")

Using snr: -1.5000000000000002
Using sigma: 1.1885022274370185
Using systematic codes
Interleaver: [ 6 15  2  4 16 17 14 19  7 11 18 13  3  8  5 10  1  0 12  9]
Encoder turbo-155-7
SystematicTurboEncoderSpec(systematic_code=<src.channelcoding.encoders.TrellisCode object at 0x7f039930d040>, interleaved_code=<src.channelcoding.encoders.GeneralizedConvolutionalCode object at 0x7f02b83d4220>)
Cross Entropy: 0.22850501537322998
Constellation Entropy: 155.32859802246094
Mean Conditional Entropy: 154.68893432617188
Input entropy: 0.6931471824645996
Entropy score estimate: 0.0534820556640625
Decoder KL Divergence estimate: 0.17502295970916748


# Case 1
Let $X, Y$ be [[Random Variable]]s so that

$$\begin{align*}
&X \sim \text{Unif}\{\pm 1\}\\
&Y \sim  X \cdot \text{Unif}[0, 1]
\end{align*}$$
Then
$$\begin{align*}
f_{Y}(y) &= \frac{1}{2} \mathbb{1}_{[0, 1]}(y) + \frac{1}{2} \mathbb{1}_{[-1,0]}(y)\\
&=\frac{1}{2}\mathbb{1}_{[-1,1]}(y),
\end{align*}$$
that is, $Y \sim \text{Unif}[-1, 1]$. By [[Differential Entropy of Continuous Uniform Random Variable]] we have that
$$\mathbb{H}(Y) = \ln (1 + 1) = \ln 2$$
By sampling we should be able to get a similar value. In particular, suppose we sample $x_{i} \in \{-1, 1\}$ and $y_{i} \in [-1, 1]$. Then we have that
$$\begin{align*}
&f_{Y|X=1} = \mathbb{1}_{[0,1]}\\
&f_{Y|X=-1} = \mathbb{1}_{[-1,0]}
\end{align*}$$


In [4]:
def unif_conditional_density_matrix(m):
    X = tf.cast(2 * tf.random.uniform((m,), minval=0, maxval=2, dtype=tf.int32) - 1, tf.float32)
    Y = X * tf.random.uniform((m,), minval=0, maxval=1, dtype=tf.float32)
    outer_prod = Y[:, None] * X[None, :]
    cond_density = tf.cast(tf.logical_and(outer_prod >= 0, outer_prod <= 1), tf.float32)
    return X, Y, cond_density

unif_conditional_density_matrix(4)

(<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1., -1., -1., -1.], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 0.25897527, -0.9543897 , -0.36819065, -0.5703981 ], dtype=float32)>,
 <tf.Tensor: shape=(4, 4), dtype=float32, numpy=
 array([[1., 0., 0., 0.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.]], dtype=float32)>)

In [5]:
sample_size = 100
Xvar, Yvar, fYX = unif_conditional_density_matrix(sample_size)
logfYX = tf.math.log(fYX)
estimate = entropy_estimate(logfYX, tf.range(sample_size))
truth = tf.math.log(2.)

print(f'Estimate: {estimate}')
print(f'Truth: {truth}')

Estimate: 0.689943790435791
Truth: 0.6931471824645996


In [6]:
sample_size = 4
Xvar, Yvar, fYX = unif_conditional_density_matrix(sample_size)
indices_where_1 = tf.reshape(tf.where(Xvar == 1.), (-1,))
print(Xvar)
print(indices_where_1)
logfYX = tf.math.log(fYX)
estimate = entropy_estimate(logfYX, indices_where_1)
truth = tf.math.log(1.)

print(f'Estimate: {estimate}')
print(f'Truth: {truth}')

tf.Tensor([ 1.  1. -1. -1.], shape=(4,), dtype=float32)
tf.Tensor([0 1], shape=(2,), dtype=int64)
Estimate: 0.0
Truth: 0.0


In [26]:
Y = tf.constant([
    [1., 2., 3.],
    [-3., -2. ,-1.]
])
X = tf.constant([
    [5., 4.],
    [-3., -1.],
    [0., 2.]
])
X = tf.transpose(X)

print(Y)
print(X)

# tf.tensordot(Y, X, axes=1)
# Y @ X
# tf.norm(Y, axis=1, keepdims=True)
get_squared_distance_matrix(X, Y)

tf.Tensor(
[[ 1.  2.  3.]
 [-3. -2. -1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 5. -3.  0.]
 [ 4. -1.  2.]], shape=(2, 3), dtype=float32)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[49.999996, 66.      ],
       [19.      , 59.      ]], dtype=float32)>

In [22]:
64 + 1 + 1

66